In [5]:
import os
import csv
import sys
import re
import pandas as pd
import openpyxl
from datetime import datetime
from collections import defaultdict

def find_cell_with_value(sheet, target_value):
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value == target_value:
                return cell.row, cell.column

    return None, None

def convert_xls_to_xlsx(input_file, output_file):
    try:
        xls_data = pd.read_excel(input_file)
        xls_data.to_excel(output_file, index=False)
    except Exception as e:
        print(f"An error occurred: {e}")

def check_difference(previous_distance, current_distance, previous_datetime, current_datetime):
    time_difference = current_datetime - previous_datetime
    time_difference_in_minutes = divmod(time_difference.total_seconds(), 60)[0]
    allowed_deviation = time_difference_in_minutes/3
    
    if time_difference_in_minutes < 1:
        if int(current_distance) - int(previous_distance) > 1:
            return 1
        else:
            return 0
        
    else:
        if int(current_distance) - int(previous_distance) > time_difference_in_minutes+allowed_deviation:
            return 1
        else:
            return 0

def calculate_difference(previous_distance, current_distance):
    return int(current_distance) - int(previous_distance)

def get_route_id(input_string, csv_filename, folder_path):
    match = re.search(r'Trip_(\d+)', folder_path)
    route_id = match.group(1)
    
    if not os.path.exists(csv_filename):
        with open(csv_filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['route_id', 'route_name'])

    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            if row['route_name'] == input_string:
                return int(row['route_id'])

    with open(csv_filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([route_id, input_string])
    
    return int(route_id)

def get_driver_id(input_string, csv_filename):
    if not os.path.exists(csv_filename):
        with open(csv_filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['driver_id', 'driver_license_id'])

    last_id = 0
    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            last_id = max(last_id, int(row['driver_id']))

    with open(csv_filename, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            if row['driver_license_id'] == input_string:
                return int(row['driver_id'])

    new_id = last_id + 1
    with open(csv_filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([new_id, input_string])

    return new_id

def append_vehicle_coordinates(data_to_append, csv_file_path):
    header = ["route_id", "driver_id"]
    existing_columns = 0
    try:
        with open(csv_file_path, 'r') as file:
            reader = csv.reader(file)
            existing_columns = len(next(reader)) - 2
    except FileNotFoundError:
        pass

    header += [str(existing_columns + i) for i in range(101)]

    with open(csv_file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        if existing_columns == 0:
            writer.writerow(header)
        writer.writerows(data_to_append)
        
def append_vehicle_status(data_to_append, csv_file_path):
    header = ["route_id", "driver_id", "start_time"]
    existing_columns = 0
    try:
        with open(csv_file_path, 'r') as file:
            reader = csv.reader(file)
            existing_columns = len(next(reader)) - 3
    except FileNotFoundError:
        pass

    header += [str(existing_columns + i) for i in range(201)]

    with open(csv_file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        if existing_columns == 0:
            writer.writerow(header)
        writer.writerows(data_to_append)
        
_last_print_len = 0 
def reprint(msg, finish=False): 
    global _last_print_len 
     
    print(' '*_last_print_len, end='\r') 
     
    if finish: 
        end = '\n' 
        _last_print_len = 0 
    else: 
        end = '\r' 
        _last_print_len = len(msg) 
     
    print(msg, end=end) 

route_id_part_of_folder_name = input("Enter route id: ")
folder_path = fr"C:\Users\TechnoPurple\Desktop\NG Trips\Trip_{route_id_part_of_folder_name}"
eligible_rows = []

if os.path.exists(f"{folder_path}\Converted Files"):
    sys.exit('Data already processed!!')

output_folder_name = 'Converted Files'
output_folder = os.path.join(folder_path, output_folder_name)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
output_folder_name_1 = 'Output Files'
output_folder_1 = os.path.join(folder_path, output_folder_name_1)
if not os.path.exists(output_folder_1):
    os.mkdir(output_folder_1)

files = os.listdir(folder_path)
trip_count = 0

for file in files:
    xls_file_path = os.path.join(folder_path, file)
    if os.path.isfile(xls_file_path):
        if file.startswith('TripTrackedReport') and file.endswith('.xls'):
            output_file = os.path.join(output_folder, f"TripTrackedReport.xlsx")
            input_file = os.path.join(folder_path, file)
            convert_xls_to_xlsx(input_file, output_file)

        else:
            trip_count += 1 
            output_file = os.path.join(output_folder, f"Trip {trip_count}.xlsx")
            input_file = os.path.join(folder_path, file)
            convert_xls_to_xlsx(input_file, output_file)

new_folder_path = f"{folder_path}\Converted Files"
        
for filename in sorted(os.listdir(new_folder_path)):
    if filename.startswith('TripTrackedReport') and filename.endswith('.xlsx'):
        file_path = os.path.join(new_folder_path, filename)
        workbook = openpyxl.load_workbook(file_path)
        report_sheet_name = 'Sheet1'
        report_sheet = workbook[report_sheet_name]

        gps_up_time_row, gps_up_time_col = find_cell_with_value(report_sheet, 'GPS Up Time')
        report_sheet.cell(row=gps_up_time_row, column=gps_up_time_col, value='GPS_Up_Time')
        gps_up_time_row, gps_up_time_col = find_cell_with_value(report_sheet, 'GPS Up Time')
        
        gps_distance_row, gps_distance_col = find_cell_with_value(report_sheet, 'GPS Distance')
        
        trip_end_reason_row, trip_end_reason_col = find_cell_with_value(report_sheet, 'Trip End Reason')
        current_row = gps_distance_row+1
        
        for row in report_sheet.iter_rows(min_row=gps_up_time_row + 1):
            gps_up_time_value = report_sheet.cell(row=current_row, column=gps_up_time_col).value
            gps_distance_value = report_sheet.cell(row=current_row, column=gps_distance_col).value
            
            trip_end_reason_value = report_sheet.cell(row=current_row, column=trip_end_reason_col).value
            gps_up_time_tv = int(gps_up_time_value)>60
            gps_distance_tv = int(gps_distance_value)>60 and int(gps_distance_value)<120
            trip_end_reason_tv = (trip_end_reason_value.lower()=='Geofence Entry/Exit'.lower())
            final_tv = gps_up_time_tv and gps_distance_tv and trip_end_reason_tv
            if gps_up_time_value is not None and gps_distance_value is not None and trip_end_reason_value is not None and final_tv:
                eligible_rows.append(row[0].row)
                
            current_row += 1

eligible_rows.sort()
total_files_count = len(eligible_rows)
total_files_processed = 0
    
for row_index in eligible_rows:
    file_index = row_index - gps_up_time_row
    filename = f'Trip {file_index}.xlsx'
    file_path = os.path.join(new_folder_path, filename)
    
    if not(os.path.exists(file_path)):
        continue
    
    workbook = openpyxl.load_workbook(file_path)

    sheet_name = 'Sheet1'
    sheet = workbook[sheet_name]

    target_value = 'Distance'

    row_index, column_index = find_cell_with_value(sheet, target_value)
    sheet.cell(row=row_index, column=column_index, value='Total Distance')
    row_index, column_index = find_cell_with_value(sheet, target_value)

    fw_row_index = row_index + 2
    fluctuation_column = column_index + 1
    actual_distance_column = column_index + 2

    sheet.cell(row=row_index, column=fluctuation_column, value='Fluctuation')
    sheet.cell(row=row_index, column=actual_distance_column, value='Actual Distance')

    sheet.cell(row=row_index + 1, column=fluctuation_column, value=0)
    sheet.cell(row=row_index + 1, column=actual_distance_column, value=0)

    if row_index and column_index:
        while sheet.cell(row=fw_row_index, column=column_index).value is not None:

            current_distance = sheet.cell(row=fw_row_index, column=column_index).value
            previous_distance = sheet.cell(row=fw_row_index - 1, column=column_index).value
            current_date = sheet.cell(row=fw_row_index, column=column_index-6).value
            previous_date = sheet.cell(row=fw_row_index - 1, column=column_index-6).value
            current_time = sheet.cell(row=fw_row_index, column=column_index-5).value
            previous_time = sheet.cell(row=fw_row_index - 1, column=column_index-5).value

            current_datetime = datetime.strptime(current_date + ' ' + current_time, '%Y-%m-%d %H:%M:%S')
            previous_datetime = datetime.strptime(previous_date + ' ' + previous_time, '%Y-%m-%d %H:%M:%S')

            difference = check_difference(previous_distance, current_distance, previous_datetime, current_datetime)
            sheet.cell(row=fw_row_index, column=fluctuation_column, value=difference)

            previous_actual_distance = sheet.cell(row=fw_row_index - 1, column=actual_distance_column).value

            if sheet.cell(row=fw_row_index, column=fluctuation_column).value == 0:
                calculated_difference = calculate_difference(previous_distance, current_distance)
                actual_distance_value = previous_actual_distance + calculated_difference
                sheet.cell(row=fw_row_index, column=actual_distance_column, value=actual_distance_value)
            else:
                sheet.cell(row=fw_row_index, column=actual_distance_column, value=previous_actual_distance)

            fw_row_index += 1

    else:
        print(f"'{target_value}' not found in the sheet.")

    selected_data = []
    for row in sheet.iter_rows(min_row=row_index+1, max_row=fw_row_index-1, min_col=0, max_col=column_index+2):
        selected_data.append([cell.value for cell in row])

    selected_data_sorted = sorted(selected_data, key=lambda x: x[0] + ' ' + x[1], reverse=True)

    unique_data_dict = defaultdict(list)
    for row in selected_data_sorted:
        date, time = row[0], row[1].split(':')[0]
        unique_data_dict[(date, time)].append(row)

    unique_data = [data[0] for data in unique_data_dict.values()]

    unique_data_sorted = sorted(unique_data, key=lambda x: x[0] + ' ' + x[1])

    selected_columns_data = [[row[0], row[1], row[4], row[8]] for row in unique_data_sorted]

    new_sheet = workbook.create_sheet(title='Master Data')
    new_sheet.append(['Date', 'Time', 'Coordinate', 'Actual Distance', 'Status', 'Driving Duration'])

    for row in selected_columns_data:
        new_sheet.append(row)

    new_target_value = 'Actual Distance'
    new_row_index, new_column_index = find_cell_with_value(new_sheet, new_target_value)

    new_sheet.cell(row=new_row_index+1, column=new_column_index+1, value='Trip Start')
    new_sheet.cell(row=new_row_index+1, column=new_column_index+2, value=0)
    sw_row_index = new_row_index + 2

    while new_sheet.cell(row=sw_row_index, column=new_column_index).value is not None:
        new_current_distance = new_sheet.cell(row=sw_row_index, column=new_column_index).value
        new_previous_distance = new_sheet.cell(row=sw_row_index - 1, column=new_column_index).value

        new_difference = calculate_difference(new_previous_distance, new_current_distance)
        driving_duration_value = new_sheet.cell(row=sw_row_index-1, column=new_column_index+2).value

        if new_difference < 5:
            new_sheet.cell(row=sw_row_index, column=new_column_index+1, value='parked')
            new_sheet.cell(row=sw_row_index, column=new_column_index+2, value=driving_duration_value)
        else:
            new_sheet.cell(row=sw_row_index, column=new_column_index+1, value='driving')
            new_sheet.cell(row=sw_row_index, column=new_column_index+2, value=driving_duration_value+1)

        sw_row_index += 1

    for row in new_sheet.iter_rows(min_row=2, max_row=new_sheet.max_row, min_col=new_column_index+2, max_col=new_column_index+2):
        if row[0].value == 1:
            first_driving_row_index = row[0].row
            break

    new_sheet.cell(row=first_driving_row_index-1, column=new_column_index+1, value='Trip Start')
    for i in range(2, first_driving_row_index-1):
        new_sheet.cell(row=i, column=new_column_index+1, value='--')

    trip_row_index, trip_column_index = find_cell_with_value(sheet, 'Trip Name')
    driver_row_index, driver_column_index = find_cell_with_value(sheet, 'Driver License')
    time_row_index, time_column_index = find_cell_with_value(new_sheet, 'Trip Start')
    
    route_name = sheet.cell(row=trip_row_index+1, column=trip_column_index).value
    route_id = get_route_id(route_name, 'route_id_data.csv', folder_path)
    driver_license_id = sheet.cell(row=driver_row_index+1, column=driver_column_index).value
    driver_id = get_driver_id(driver_license_id, f"{output_folder_1}\driver_id_data.csv")
    start_time_hhmmss = new_sheet.cell(row=time_row_index, column=time_column_index-3).value
    start_time = start_time_hhmmss[:2]
    last_row = new_sheet.max_row

    new_selected_data = []
    for row_number in range(time_row_index+1, last_row + 1):
        cell_value = new_sheet.cell(row=row_number, column=time_column_index).value
        new_selected_data.append((cell_value))

    status_data_to_append = []
    status_data_to_append.append([route_id, driver_id, start_time, 'driving'] + new_selected_data)

    coordinates_data = []
    for row in new_sheet.iter_rows(min_row=2, values_only=True):
        driving_duration = row[5]
        if row[4] == "driving":
            coordinates = row[2]
            coordinates_data.append(coordinates)

    coordinates_data_to_append = []
    coordinates_data_to_append.append([route_id, driver_id] + coordinates_data)

    workbook.save(file_path)
    
    vehicle_status_path = f"{output_folder_1}\VehicleStatus.csv"
    append_vehicle_status(status_data_to_append, vehicle_status_path)
    
    vehicle_coordinates_path = f"{output_folder_1}\VehicleCoordinates.csv"
    append_vehicle_coordinates(coordinates_data_to_append, vehicle_coordinates_path)
    
    total_files_processed += 1
    reprint(f"{total_files_processed}/{total_files_count} processed")

print("Task completed.")

Enter route id: 1004
Task completed.
